In [5]:
from agno.agent import Agent
from agno.models.openai.like import OpenAILike
from agno.tools.reasoning import ReasoningTools
from agno.tools import tool
from agno.tools.yfinance import YFinanceTools
import random
import logging

# logging.basicConfig(format='%(asctime)s [%(levelname)s] %(name)s:%(filename)s:%(lineno)d - %(message)s',datefmt='%Y-%m-%d %H:%M:%S',level=logging.DEBUG) 

#------------------ settings ------------------
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
local_base_url = 'http://192.168.0.166:8000/v1'
local_model_name = 'Qwen3-235B'
model_name = 'qwen-plus-latest'

local_settings = {
  'api_key' : '123',
  'base_url' : local_base_url,
  'id' : local_model_name
}

qwen_settings = {
  'api_key' : api_key,
  'base_url' : base_url,
  'id' : model_name
}

settings = qwen_settings
#------------------ settings ------------------

## 1. Most common custom tool write by yourself

In [32]:
@tool(show_result=True)
def get_weather_by_city(city:str)->str:
    '''
    Get the weather of a city.
    
    Args:
        city (str): The name of the city.
        
    Returns:
        str: The weather of the city.
    '''
    weather_choice = ["Sunny", "Cloudy", "Rainy"]
    return random.choice(weather_choice)

### Parameters meanings:

- `show_tool_calls` in `Agent` Class: only used in `run_response` method.
- `show_result` in decorator `@tool(show_result=True)` maybe used in `run_response` method.

In [33]:
agent = Agent(
    model=OpenAILike(**settings),
    tools=[get_weather_by_city],
    description="You are an helpful assistant.",
    instructions=["Reply concisely."],
    debug_mode=False,
    telemetry=False,
)

In [34]:
response = agent.run(message='What is the weather of changsha today?')

In [35]:
response.content

'RainyThe weather in Changsha today is rainy.'

In [36]:
response.messages

[Message(role='system', content='You are an helpful assistant.\n<instructions>\nReply concisely.\n</instructions>', name=None, tool_call_id=None, tool_calls=None, audio=None, images=None, videos=None, files=None, audio_output=None, image_output=None, thinking=None, redacted_thinking=None, provider_data=None, citations=None, reasoning_content=None, tool_name=None, tool_args=None, tool_call_error=None, stop_after_tool_call=False, add_to_agent_memory=True, from_history=False, metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, audio_tokens=0, input_audio_tokens=0, output_audio_tokens=0, cached_tokens=0, cache_write_tokens=0, reasoning_tokens=0, prompt_tokens=0, completion_tokens=0, prompt_tokens_details=None, completion_tokens_details=None, additional_metrics=None, time=None, time_to_first_token=None, timer=None), references=None, created_at=1748852346),
 Message(role='user', content='What is the weather of changsha today?', name=None, tool_call_id=None, tool_calls=Non

In [37]:
Response_iter = agent.run(message='What is the weather of changsha today?',stream=True)
for response in Response_iter:
    print(response.event,response)

ToolCallStarted RunResponse(content=None, content_type='str', thinking=None, reasoning_content=None, event='ToolCallStarted', messages=None, metrics=None, model='qwen-plus-latest', model_provider=None, run_id='9da74c46-e016-439d-b480-1c96ea83e7e4', agent_id='746b0a8c-384e-4734-abba-6e6fc690bdd5', session_id='b0567dfb-4182-4c88-a531-fe85d2458402', workflow_id=None, tools=[ToolExecution(tool_call_id='call_06b2df0bab2344228371e7', tool_name='get_weather_by_city', tool_args={'city': 'changsha'}, tool_call_error=None, result=None, metrics=None, stop_after_tool_call=False, created_at=1748739810, requires_confirmation=None, confirmed=None, confirmation_note=None, requires_user_input=None, user_input_schema=None, external_execution_required=None)], formatted_tool_calls=None, images=None, videos=None, audio=None, response_audio=None, citations=None, extra_data=None, created_at=1748739810)
RunResponse RunResponse(content='Cloudy', content_type='str', thinking=None, reasoning_content=None, event=

## 2. Toolkit Class

- through some parameters provided by `Toolkit` Class such as `stop_after_tool_call_tools=["google_search"]` to control the behavior of the Agent.

In [ ]:
from agno.tools.googlesearch import GoogleSearchTools

# agent = Agent(model=OpenAILike(**settings),tools=[GoogleSearchTools(stop_after_tool_call_tools=["google_search"],)],telemetry=False)
agent = Agent(model=OpenAILike(**settings),tools=[GoogleSearchTools()],telemetry=False)

In [45]:
response = agent.run('Is Elon Musk still in government has a position?')

In [46]:
response.content

''

In [47]:
response.messages

[Message(role='user', content='Is Elon Musk still in government has a position?', name=None, tool_call_id=None, tool_calls=None, audio=None, images=None, videos=None, files=None, audio_output=None, image_output=None, thinking=None, redacted_thinking=None, provider_data=None, citations=None, reasoning_content=None, tool_name=None, tool_args=None, tool_call_error=None, stop_after_tool_call=False, add_to_agent_memory=True, from_history=False, metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, audio_tokens=0, input_audio_tokens=0, output_audio_tokens=0, cached_tokens=0, cache_write_tokens=0, reasoning_tokens=0, prompt_tokens=0, completion_tokens=0, prompt_tokens_details=None, completion_tokens_details=None, additional_metrics=None, time=None, time_to_first_token=None, timer=None), references=None, created_at=1748853299),
 Message(role='assistant', content='', name=None, tool_call_id=None, tool_calls=[{'id': 'call_52c5a4cf558e4ac3b2bb7f', 'function': {'arguments': '{"la

## 3. Write your own custom tools

1. use `tool` decorator to create a custom tool function
2. 